### Analysis_5
If a user likes one movie, recommend five movies that the usesr might also like<br>

Steps:<br>
Take movieId1 as an example
1. Find all users like movieId1 (if a user rates movieId1 >= his/her own average score, then consider him/her as like this movie)
2. Given all users from step1, find out all other movies they like
3. Based on movie's genre, the times they were liked, tmdb keyword relavance(fetch data from tmdb api), movie average score, calculate a final result

In [1]:
import os
import pandas as pd
from pandas import DataFrame as df
from datetime import datetime
import time
from collections import Counter
import json
import requests

In [2]:
key = os.getenv('auth_key')

In [3]:
rootdir = os.path.expanduser('~')+'/final/data'

In [4]:
ratings = pd.read_csv(rootdir+'/ml-20m-dataset/ratings.csv')
movies = pd.read_csv(rootdir+'/ml-20m-dataset/movies.csv')
tags = pd.read_csv(rootdir+'/ml-20m-dataset/tags.csv')
links = pd.read_csv(rootdir+'/ml-20m-dataset/links.csv')

**I need to find an example**<br>
I want to choose a movie from movies that have 20 ratings and average rating is 3. <br>
Among these movies, I prefer the ones with bigger standard deviation <br>
I decided to go for "Lan Yu" (only because it's the same pronunciation as one of my favorite teams in my favorite novel :) )

In [5]:
movies['year'] = movies['title'].apply(lambda x: x[-5:-1])

In [6]:
year_filter = movies['title'].apply(lambda x: True if x[-5:-1].isdigit() else False)

In [7]:
movies = movies[year_filter]

In [8]:
movies.head(3)

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995


In [9]:
a = ratings.groupby('movieId').count().reset_index()

In [10]:
b = ratings[['movieId','rating']].merge(a[a['userId']==20][['movieId']],on='movieId',how='inner')[['movieId','rating']].groupby('movieId').mean().round().reset_index()

In [11]:
b[b['rating']==3].shape

(93, 2)

In [12]:
ratings[['movieId','rating']].merge(b[b['rating']==3][['movieId']],on='movieId')[['movieId','rating']].groupby('movieId').std().sort_values(by='rating').tail(5)

,rating
movieId,
49735,1.336482
983,1.386969
5514,1.409320
109420,1.520691
6026,1.532756


In [13]:
movies[movies['movieId']==5514]

,movieId,title,genres,year
5417,5514,Lan Yu (2001),Drama|Romance,2001


**Find out users who liked movieId 5514**<br>
For each and every user that rated movieId 5514, find out if the rating is above this user's average rating.<br>
If yes, then consider this user likes movieId 5514, otherwise ignore. <br>

In [14]:
c = ratings[ratings['movieId']==5514].reset_index()[['userId','rating']]

In [15]:
user_avgRating = c['userId'].apply(lambda x: ratings[ratings['userId']==x]['rating'].mean())

In [16]:
c['user_avg'] = user_avgRating

In [17]:
user_like_5514 = c[c['rating']>c['user_avg']]

In [18]:
user_like_5514.reset_index()

,index,userId,rating,user_avg
0,0,7571,4.0,3.529274
1,2,49285,4.0,3.385081
2,4,53640,4.5,3.349975
3,5,53853,3.5,2.862251
4,6,53967,4.0,3.661082
5,9,65043,4.0,3.213178
6,13,76992,3.5,3.478659
7,15,88785,4.0,3.687766
8,16,91583,4.0,3.890430


**Find out all other movies liked by these users**

In [19]:
movie_list = []

In [20]:
def find_like_movies(user_id):
    user_avg = user_like_5514['user_avg'][user_like_5514['userId']==user_id].values[0]
    df_temp = ratings[ratings['userId']==user_id]
    list_temp = df_temp[df_temp['rating'].apply(lambda x: x > user_avg)]['movieId'].values
    for x in list_temp:
        movie_list.append(x)

In [21]:
user_like_5514['userId'].apply(find_like_movies)

0     None
2     None
4     None
5     None
6     None
9     None
13    None
15    None
16    None
Name: userId, dtype: object

In [22]:
len(movie_list)

4367

In [23]:
len(set(movie_list))

2864

In [24]:
counts = Counter(movie_list)

In [25]:
counts.most_common(10)

[(5514, 9),
 (7361, 7),
 (1247, 6),
 (2692, 6),
 (2997, 6),
 (4034, 6),
 (4226, 6),
 (4308, 6),
 (4993, 6),
 (6016, 6)]

**Calculate movie score** <br>
Use movie's genre, times it being liked, imbd keyword relevance, and movie average rating to find out top five movies that might interest the user. <br>
I haven't learn algorithm or machine learning before, so I'd just make up some percentage and points for this analysis as of now.<br>
MovieReleveceScore = 20(genres) + 30(times it being liked) + 30(keyword relevance) + 20(movie avg rating)

**Create new dataframe**

In [26]:
series_id = pd.Series(data=list(counts.keys()))

In [27]:
series_id.shape

(2864,)

In [28]:
df_score = series_id.reset_index().rename(columns={0:'movieId'})

In [29]:
df_score = df_score.drop(['index'],axis=1)

In [30]:
df_score.head(2)

,movieId
0,1
1,8195


In [31]:
series_occurrence = df_score['movieId'].apply(lambda x: counts[x])

In [32]:
df_score['like_occurence'] = series_occurrence

In [33]:
df_score = df_score.merge(movies[['movieId','title','genres']],on='movieId',how='inner')

In [34]:
df_score.head(3)

,movieId,like_occurence,title,genres
0,1,4,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,8195,1,"Avventura, L' (Adventure, The) (1960)",Drama|Mystery|Romance
2,6,3,Heat (1995),Action|Crime|Thriller


**Part1: genre**<br>
Drama|Romance <br>
If a movie has exactly these two genres, gets 20 points.<br>
If a moive miss one of them, - 10 points each <br>
If a movie has irrelevant genres, - 1 point each<br>
Min point is 0

In [35]:
def get_genre_score(genres):
    # if only one genre
    if '|' not in genres:
        if genres=='Drama' or genres=='Romance':
            return 10
        else: return 0
    # if two genres
    else: 
        if len(genres.split('|'))==2:
            if 'Drama' in genres:
                if 'Romance' in genres:
                    return 20
                else: return 9
            else:
                if 'Romance' in genres:
                    return 9
                else: return 0
    # if more than two genres
        else:
            if 'Drama' in genres:
                if 'Romance' in genres:
                    return 22-len(genres.split('|'))
                else: return 11-len(genres.split('|'))
            else:
                if 'Romance' in genres:
                    return 11-len(genres.split('|'))
                else: return 0

In [36]:
df_score['genre_score'] = df_score['genres'].apply(get_genre_score)

In [37]:
df_score.head(3)

,movieId,like_occurence,title,genres,genre_score
0,1,4,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0
1,8195,1,"Avventura, L' (Adventure, The) (1960)",Drama|Mystery|Romance,19
2,6,3,Heat (1995),Action|Crime|Thriller,0


**Part2: times it being liked**<br>
Since there are max 9 users like this movie, score=10\*(times_liked/3)

In [38]:
df_score['like_score'] = df_score['like_occurence'].apply(lambda x: (10*x/3))

In [39]:
df_score.head(3)

,movieId,like_occurence,title,genres,genre_score,like_score
0,1,4,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,13.333333
1,8195,1,"Avventura, L' (Adventure, The) (1960)",Drama|Mystery|Romance,19,3.333333
2,6,3,Heat (1995),Action|Crime|Thriller,0,10.000000


**Part3: tmdb keyword relevance**<br>
fetch data first

In [40]:
keyword_data = rootdir+'/tmdb_movie_keyword_data/'

In [41]:
df_tmdbid = links[['movieId','tmdbId']].merge(df_score[['movieId']],on='movieId',how='inner')

In [42]:
df_tmdbid.shape

(2864, 2)

Seems like I have two NaN to take care, assign 0 to these two ids

In [43]:
df_tmdbid['tmdbId'] = df_tmdbid['tmdbId'].fillna(0.0).astype(int)

In [44]:
df_tmdbid[df_tmdbid['tmdbId']==0]

,movieId,tmdbId
194,720,0
2762,87370,0


In [45]:
df_tmdbid[df_tmdbid['movieId']==5514]

,movieId,tmdbId
1347,5514,27062


In [46]:
def id_file_name(tmdb_id):
    file_name = keyword_data+'id_'+str(tmdb_id)+'.json'
    return file_name

In [ ]:
error_list = []
for tmdb_id in df_tmdbid['tmdbId']:
    r = requests.get('https://api.themoviedb.org/3/movie/' + str(tmdb_id) + '/keywords?api_key=' + key)
    if r.status_code==requests.codes.ok:
        file = id_file_name(tmdb_id)
        with open(file, 'w') as fp:
            json.dump(r.json(),fp)
        time.sleep(0.7)
    else:
        print("Error occured when processing id",tmdb_id,"Error code:",r.status_code)
        error_list.append(tmdb_id)
        print(r.headers)
        time.sleep(0.7)

Error code 404, ignore them for now

In [49]:
json.loads(open(keyword_data+'id_27062.json').read())['keywords']

[{'id': 478, 'name': 'china'}, {'id': 10198, 'name': 'gay relationship'}]

For each movie, read corresponding json file.<br>
If the movie contains both keywords: 30 points. <br>
If the movie contains only one keyword: 20 points. <br>
If none of the keywords were found: 0 points. <br>

In [50]:
def keyword_score(tmdbid):
    temp_id_list = [] 
    try:
        for record in json.loads(open(keyword_data + 'id_' + str(tmdbid) + '.json').read())['keywords']:
            temp_id_list.append(record['id'])
    except FileNotFoundError:
        print('File for id_', tmdbid,' not found')
    if 478 in temp_id_list:
        if 10198 in temp_id_list:
            return 30
        else: return 20
    else:
        if 10198 in temp_id_list:
            return 20
        else: return 0

In [51]:
series_keyword_score = df_tmdbid['tmdbId'].apply(keyword_score)

File for id_ 0  not found
File for id_ 253768  not found
File for id_ 0  not found
File for id_ 58423  not found


In [52]:
df_score['keyword_score'] = series_keyword_score

In [53]:
df_score.head()

,movieId,like_occurence,title,genres,genre_score,like_score,keyword_score
0,1,4,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,13.333333,0
1,8195,1,"Avventura, L' (Adventure, The) (1960)",Drama|Mystery|Romance,19,3.333333,0
2,6,3,Heat (1995),Action|Crime|Thriller,0,10.000000,0
3,7,1,Sabrina (1995),Comedy|Romance,9,3.333333,0
4,51884,1,"Namesake, The (2006)",Drama|Romance,20,3.333333,0


**Part4: movie avg rating**<br>
Score = 20\*(avg_rating/5)

In [54]:
df_score = df_score.merge(ratings[['movieId','rating']].groupby('movieId').mean().reset_index(),on='movieId',how='inner')

In [55]:
df_score.head(3)

,movieId,like_occurence,title,genres,genre_score,like_score,keyword_score,rating
0,1,4,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,13.333333,0,3.921240
1,8195,1,"Avventura, L' (Adventure, The) (1960)",Drama|Mystery|Romance,19,3.333333,0,3.949074
2,6,3,Heat (1995),Action|Crime|Thriller,0,10.000000,0,3.834930


In [56]:
df_score['rating_score'] = df_score['rating'].apply(lambda x: 4*x)

In [57]:
df_score.head()

,movieId,like_occurence,title,genres,genre_score,like_score,keyword_score,rating,rating_score
0,1,4,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,13.333333,0,3.921240,15.684958
1,8195,1,"Avventura, L' (Adventure, The) (1960)",Drama|Mystery|Romance,19,3.333333,0,3.949074,15.796296
2,6,3,Heat (1995),Action|Crime|Thriller,0,10.000000,0,3.834930,15.339721
3,7,1,Sabrina (1995),Comedy|Romance,9,3.333333,0,3.366484,13.465936
4,51884,1,"Namesake, The (2006)",Drama|Romance,20,3.333333,0,3.734216,14.936864


**Part5: Calculate total score**

In [58]:
df_score['total'] = df_score.genre_score+df_score.like_score+df_score.keyword_score+df_score.rating_score

In [59]:
df_score.sort_values(by='total',ascending=False)[['title','genres']].head()

,title,genres
1347,Spanglish (2004),Comedy|Drama|Romance
1562,Happy Together (a.k.a. Buenos Aires Affair) (C...,Drama|Romance
2206,"Pianist, The (2002)",Drama|War
104,Eat Drink Man Woman (Yin shi nan nu) (1994),Comedy|Drama|Romance
137,Lan Yu (2001),Drama|Romance
